# Data mining from PubChem 

In [14]:
import json
import numpy as np
import pandas as pd

In [ ]:
import os 
import sys
import time
import json

import os, shutil
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio.PDB import *
from biopandas.pdb import PandasPdb
from Bio import ExPASy

In [15]:
# Timing function
def task_begin():
    import time
    since = time.time()
    print('The task is being executed...\n----------\n')
    return since

def task_done(since):
    import time
    print('\n----------\nThe task has been done.')
    time_elapsed = time.time() - since
    print('Time cost {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

In [16]:
# the web address of Tested Compounds in PubChem
link = 'https://pubchem.ncbi.nlm.nih.gov/sdq/cgi2rcgi.cgi?infmt=json&outfmt=csv&query={%22download%22:%22*%22,%22collection%22:%22bioactivity%22,%22where%22:{%22ands%22:[{%22protacxn%22:%22notnull%22},{%22cid%22:%22notnull%22},{%22repacxn%22:%22ProID%22}]},%22order%22:[%22acvalue,asc%22],%22start%22:1,%22limit%22:10000000,%22downloadfilename%22:%22PROTACXN_ProID_bioactivity_protein%22}'

In [17]:
# DDR1 UniProt ID is Q08345
pro_id = 'Q08345'
link1 = link.replace('ProID', pro_id)

In [18]:
since = task_begin()

for i in range(2):
    try:
        data = pd.read_csv(link1)
        data.to_csv(pro_id + '_Tested_Compounds_PubChem.csv', index=False)
        print(f'The test compounds of {pro_id} has been saved as {pro_id}_Tested_Compounds.csv.')
        break
    except Exception as a:
        print(str(a)+" is the error , Trying {} time".format(i))
        continue
    else:
        break
else:
    print("something Wrong , Try running Again [refer error code for more]")
    
task_done(since)

The task is being executed...
----------

The test compounds of Q08345 has been saved as Q08345_Tested_Compounds.csv.

----------
The task has been done.
Time cost 0m 8s


In [19]:
testedCompound = pd.read_csv(pro_id + "_Tested_Compounds_PubChem.csv", error_bad_lines=False)

In [20]:
testedCompound

,baid,acvalue,aid,sid,cid,geneid,pmid,aidtype,aidmdate,hasdrc,...,acname,acqualifier,aidsrcname,aidname,cmpdname,targetname,targeturl,ecs,repacxn,taxids
0,70868681,NaN,1982,48409448,2858522,780,NaN,Other,20110103,0,...,NaN,NaN,Burnham Center for Chemical Genomics,Kinase inhibition selectivity assay for compou...,"1-(3,5-Ditert-butyl-4-hydroxyphenyl)-2-[2-(3-h...",DDR1 - discoidin domain receptor tyrosine kina...,/gene/780,2.7.10.1,Q08345,NaN
1,70869002,NaN,1982,48409448,2858522,780,NaN,Other,20110103,0,...,NaN,NaN,Burnham Center for Chemical Genomics,Kinase inhibition selectivity assay for compou...,"1-(3,5-Ditert-butyl-4-hydroxyphenyl)-2-[2-(3-h...",DDR1 - discoidin domain receptor tyrosine kina...,/gene/780,2.7.10.1,Q08345,NaN
2,70869323,NaN,1982,48409448,2858522,780,NaN,Other,20110103,0,...,NaN,NaN,Burnham Center for Chemical Genomics,Kinase inhibition selectivity assay for compou...,"1-(3,5-Ditert-butyl-4-hydroxyphenyl)-2-[2-(3-h...",DDR1 - discoidin domain receptor tyrosine kina...,/gene/780,2.7.10.1,Q08345,NaN
3,70869644,NaN,1982,48409448,2858522,780,NaN,Other,20110103,0,...,NaN,NaN,Burnham Center for Chemical Genomics,Kinase inhibition selectivity assay for compou...,"1-(3,5-Ditert-butyl-4-hydroxyphenyl)-2-[2-(3-h...",DDR1 - discoidin domain receptor tyrosine kina...,/gene/780,2.7.10.1,Q08345,NaN
4,70869965,NaN,1982,48409448,2858522,780,NaN,Other,20110103,0,...,NaN,NaN,Burnham Center for Chemical Genomics,Kinase inhibition selectivity assay for compou...,"1-(3,5-Ditert-butyl-4-hydroxyphenyl)-2-[2-(3-h...",DDR1 - discoidin domain receptor tyrosine kina...,/gene/780,2.7.10.1,Q08345,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3838,467869769,66.000,1347183,354327400,117920477,780,NaN,Literature-derived,20191216,0,...,IC50,NaN,BindingDB,LanthaScreen Eu Kinase Binding Assay in Benzam...,CSc1ccc(F)cc1CNC(=O)c1cccc(c1)C(F)(F)F,DDR1 - discoidin domain receptor tyrosine kina...,/gene/780,2.7.10.1,Q08345,NaN
3839,468973025,71.484,1424972,160678787,46216796,780,29191878.0,Confirmatory,20200620,0,...,Kd apparent,=,ChEMBL,"Kinobeads (epsilon), multiple immobilized ATP-...",Pacritinib,DDR1 - discoidin domain receptor tyrosine kina...,/gene/780,2.7.10.1,Q08345,9606.0
3840,450588714,100.000,1242618,242115264,4426513,780,26191369.0,Confirmatory,20181009,0,...,IC50,>,ChEMBL,Inhibition of DDR1 (unknown origin) after 1 hr...,1-(2-Chlorophenyl)-3-(pyridin-3-ylmethyl)urea,DDR1 - discoidin domain receptor tyrosine kina...,/gene/780,2.7.10.1,Q08345,9606.0
3841,467869483,119.000,1347183,354328551,132274453,780,NaN,Literature-derived,20191216,0,...,IC50,NaN,BindingDB,LanthaScreen Eu Kinase Binding Assay in Benzam...,N'-(5-Chloro-2-ethylsulfonylphenyl)-2-methyl-6...,DDR1 - discoidin domain receptor tyrosine kina...,/gene/780,2.7.10.1,Q08345,NaN


In [21]:
#data cleaning up
key_data = testedCompound[['cid','activity']] # get 'compound CID' and 'activity'
key_data = key_data.dropna() # remove null items
key_data.reset_index(drop=True, inplace=True)
print(f'There are {len(key_data)} compounds in all.')
key_data

There are 3843 compounds in all.


,cid,activity
0,2858522,Inactive
1,2858522,Inactive
2,2858522,Inactive
3,2858522,Inactive
4,2858522,Inactive
...,...,...
3838,117920477,Active
3839,46216796,Active
3840,4426513,Unspecified
3841,132274453,Active


In [22]:
# Select the cid with "Activity" activity and use set to reduce weight
cid_active = list(set(key_data[key_data.activity == 'Active'].cid.to_list()))
print(len(cid_active))

652


In [23]:
import pubchempy as pcp

def get_smiles(cid):
    c = pcp.Compound.from_cid(cid)
    return c.canonical_smiles

In [24]:
since = task_begin()

DDR1_smis = list(map(get_smiles, cid_active))
    
task_done(since)

The task is being executed...
----------


----------
The task has been done.
Time cost 17m 20s


In [25]:
with open('DDR1_active_smi_PubChem.json', 'w') as f:
    json.dump(DDR1_smis, f)